In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests                   
from bs4 import BeautifulSoup     
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import tika
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from tika import parser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from urllib.request import urlretrieve



In [2]:

def get_page_links(start, end):
    
    """ create links for all the pages of icobench.com"""
    
    page_links=list()
    #need till 468/469
    p=start
    while(p<=end):
        page_url=f"https://icobench.com/icos?page={p}"
        page_links.append(page_url)
        p+=1
    return page_links

# page_links = get_page_links(1, 200)
# page_links

In [3]:
def get_ico_info(page_urls):
    
    """Scraper to extract all the details/features of the coins(except Whitepaper Texts)
       on each page"""
    
    ico_links = []
#     ico_names = []
    ico_data = []
    whitepaper_links = []
    for url in page_urls:
        page = requests.get(url)
        if page.status_code == 200:
            soup = BeautifulSoup(page.content, 'html.parser')
            ico_tags = soup.select("a.name.notranslate")
            for ico in ico_tags:
                ico_name = ico.get_text().strip()
                # some problem with below two coins
                if(ico_name == "TillBilly" or ico_name == "Ink Labs Foundation"):
                    continue
                ico_link = 'https://icobench.com' + ico['href']
                ico_links.append(ico_link)
                ico_page = requests.get(ico_link)
                time.sleep(4)
                if page.status_code == 200:
                    try:
                        soup1 = BeautifulSoup(ico_page.content, 'html.parser')
                    except:
                        print(f"Soup Error in ICO:{ico_name} at pageno:{url}")
                        continue
                    try:
                        cat = ""
                        categories = soup1.select("div.ico_information div.categories a")
                        
                        for i in categories:
                            cat += i.text.strip()+","
                        cat = cat[:-1]
                        if cat == "" or len(cat) == 0:
                            cat = None
                    except:
                        cat = None
                            
                        
                    try:
                        rating = None
                        rating = soup1.select('div.fixed_data div.rate')
                        if rating and rating != []:
                            rating = rating[0].text
                    #     print("Rating",rating)
                    except:
                        rating = None
                    try:
                        preicoprice = None
                        price = None
                        fin_info = soup1.select("div.fixed_data div.financial_data div.data_row div.col_2")
                        for i in range(0, len(fin_info), 2):
                            if(fin_info[i].text.strip() == "PreICO Price" or fin_info[i].text.strip() == "Price in preICO" ):
                                preicoprice = fin_info[i+1].text.strip()
                            if(fin_info[i].text.strip() == "Price" or fin_info[i].text.strip() == "Price in ICO"):
                                price = fin_info[i+1].text.strip()
                                
                    except:
                        preicoprice = None
                        price = None
                        
                    try:
                        
                        softcap = None
                        hardcap = None
                        Raised = None
                        cap = soup1.select("div.tab_content#financial div.box_right div.row div")
                        for i in range(0, len(cap), 2):
                            if(cap[i].text.strip() == "Soft cap"):
                                softcap = cap[i+1].text 
                            if(cap[i].text.strip() == "Hard cap"):
                                hardcap = cap[i+1].text
                            if(cap[i].text.strip() == "Raised"):
                                Raised = cap[i+1].text.strip()
                         
                    except:
                        softcap = None
                        hardcap = None
                        Raised = None
                    
                    try:
                        wp_link = None
                        wp_link = soup1.select('div.tab_content#whitepaper p object div a')
                        if wp_link == None or len(wp_link)<=0:
                            wp_link = None
                        else:
                            wp_link = wp_link[0]['href']
                    except:
                        wp_link = None
                        
                    whitepaper_links.append((ico_name, wp_link))
                    ico_data.append((ico_name, cat, rating, preicoprice, price, softcap, hardcap, Raised, wp_link))
                else:
                    print(f"cannot open ico:{ico_link}")
        
                
        else:
            print(f"Cannot open pageno:{url}")
            
    return ico_data, whitepaper_links    
             
#icoprice, preicoprice, softcap, hardcap, wp__link remove blanks
# for Raised cap remove blanks and "Unknown"

In [12]:
def get_wptext(urllist):
    """To extract whitepapers of the coins"""
    wp_text_list = []
   
    for coin, url in urllist:
        wp_text = None
        if url == "" or url == None:
            continue
        try:
            time.sleep(4)
            parsed = parser.from_file(url)
            
            if(type(parsed["content"]) == str):
                wp_text = parsed["content"].strip()
                print(f"GOT Whitepaper for: {coin} ")

        except:
            wp_text = "No Access"
            print(f"Cannot parse whitepaper of {coin} at url:{url}")
        
        wp_text_list.append((coin, wp_text))
    
    return wp_text_list
    
#remove ones with "No Access" and 
# remove blanks

In [13]:
page_urls2 = get_page_links(1, 2) #test

page_urls150 = get_page_links(1, 150)
# print(page_urls)
page_urls300 = get_page_links(151, 300)

page_urls469 = get_page_links(301,469)

In [14]:
data_ICO1, whtpapr_links1 = get_ico_info(page_urls2)
# getting coin info and whitepaper links for page #1 to #150 

In [15]:
whitepaper_list1 = get_wptext(whtpapr_links1)
# extracting whitepaper text

2019-12-03 20:26:25,251 [MainThread  ] [INFO ]  Retrieving https://buyanylight.com/papers/Whitepaper_1.0.pdf?ver=20191003a to C:\Users\shiva\AppData\Local\Temp/https-buyanylight-com-papers-whitepaper_1-0-pdf-ver-20191003a.


Cannot parse whitepaper of BuyAnyLight (BAL) (PreICO) at url:https://buyanylight.com/papers/Whitepaper_1.0.pdf?ver=20191003a


2019-12-03 20:26:29,415 [MainThread  ] [INFO ]  Retrieving https://mindsync.ai/docs/whitepaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-mindsync-ai-docs-whitepaper-pdf.


Cannot parse whitepaper of Mindsync at url:https://mindsync.ai/docs/whitepaper.pdf


2019-12-03 20:26:33,537 [MainThread  ] [INFO ]  Retrieving https://snbtoken.io/wp.pdf to C:\Users\shiva\AppData\Local\Temp/https-snbtoken-io-wp-pdf.


...

Cannot parse whitepaper of DeepCloud at url:https://www.deepcloudai.com/paper/Whitepaper_DeepCloud_AI_Version1_00_ENG.pdf


2019-12-03 20:30:22,996 [MainThread  ] [INFO ]  Retrieving https://emanate.live/pdf/mn8-whitepaper-v14.pdf to C:\Users\shiva\AppData\Local\Temp/https-emanate-live-pdf-mn8-whitepaper-v14-pdf.


GOT Whitepaper for: Emanate 


2019-12-03 20:30:27,924 [MainThread  ] [INFO ]  Retrieving https://kubitx.io/wp.pdf to C:\Users\shiva\AppData\Local\Temp/https-kubitx-io-wp-pdf.


Cannot parse whitepaper of KuBitX at url:https://kubitx.io/wp.pdf


In [7]:
df_wplinks1 = pd.DataFrame(whtpapr_links1, columns = ['Coin name', "whitepaper link"]) 
df_wplinks1
#whitepaper links dataframe

,Coin name,whitepaper link
0,BuyAnyLight (BAL) (PreICO),https://buyanylight.com/papers/Whitepaper_1.0....
1,GRAYLL,https://grayll.io/wp-content/uploads/2019/06/G...
2,Mindsync,https://mindsync.ai/docs/whitepaper.pdf
3,PointPay,https://cdn.pointpay.io/WhitePaper__en.pdf?cac...
4,Tycoon,https://www.tycoon.io/whitepaper.pdf
5,Bethereum,https://www.bethereum.com/Bethereum-Whitepaper...
6,BitWings,https://bitwings.org/white-paper-en.pdf
7,Freelanex,https://www.freelanex.io/wp-content/uploads/20...
8,Global Crypto Alliance,https://gcalliance.io/wp-content/uploads/GCA_C...
9,Max Crowdfund,https://maxcrowdfund.com/sites/default/files/M...


In [9]:
df1 = pd.DataFrame(data_ICO1, columns=["CoinName","Category", "Rating","PreICO Price", "Price", "Softcap", "Hardcap", "Raised cap", "Whitepaper_link"] )
df1
#ICO coin details dataframe

,CoinName,Category,Rating,PreICO Price,Price,Softcap,Hardcap,Raised cap,Whitepaper_link
0,BuyAnyLight (BAL) (PreICO),"Electronics,Big Data,Artificial Intelligence,E...",4.8,1 BAL Token = 0.2 USD,1 BAL Token = 0.3033 USD,"4,000,000 USD","8,000,000 USD",None,https://buyanylight.com/papers/Whitepaper_1.0....
1,GRAYLL,"Cryptocurrency,Big Data,Artificial Intelligenc...",4.4,1 GRX = 0.007 USD,1 GRX = 0.01 USD,"1,500,000 USD","35,000,000 USD",None,https://grayll.io/wp-content/uploads/2019/06/G...
2,Mindsync,"Artificial Intelligence,Big Data,Business serv...",4.6,None,1 MAI = 0.14 USD,"4,200,000 USD","9,800,000 USD","$4,900,000",https://mindsync.ai/docs/whitepaper.pdf
3,PointPay,"Platform,Investment,Internet,Infrastructure,Cr...",4.6,1 PXP = 0.05 USD,1 PXP = 0.1 USD,"1,000,000 USD","30,000,000 USD",None,https://cdn.pointpay.io/WhitePaper__en.pdf?cac...
4,Tycoon,Platform,4.6,None,1 Tycoon Token / TYC = 0.1 USD,None,None,None,https://www.tycoon.io/whitepaper.pdf
5,Bethereum,"Entertainment,Cryptocurrency,Casino & Gambling...",3.9,"1 ETH = 22,750 BTHR",1 BETHER = 0.0000571429 ETH,None,"25,000 ETH",None,https://www.bethereum.com/Bethereum-Whitepaper...
6,BitWings,"Electronics,Software",4.4,1 BWN = 0.1 USD,1 BWN = 0.2 USD,"3,000,000 USD","30,000,000 USD",None,https://bitwings.org/white-paper-en.pdf
7,Freelanex,"Cryptocurrency,Platform,Smart Contract",4.5,None,1 FLXC = 0.004 USD,"1,000,000 USD","10,000,000 USD",None,https://www.freelanex.io/wp-content/uploads/20...
8,Global Crypto Alliance,Cryptocurrency,4.9,None,1 CALL = 0.02 USD,None,None,None,https://gcalliance.io/wp-content/uploads/GCA_C...
9,Max Crowdfund,"Business services,Investment,Platform,Real estate",4.4,None,1 MPG = 0.01 EUR,"500,000 EUR","5,000,000 EUR","$750,000",https://maxcrowdfund.com/sites/default/files/M...


In [16]:
wp_df1 = pd.DataFrame(whitepaper_list1, columns=["CoinName", "Whitepaper Text"])
#whitepaper text dataframe
wp_df1["Whitepaper Text"][9]
#Example whitepaper text

'WHITEPAPERpointpay.io\n\nAll-in-one solution \nfor cryptocurrency ecosystem\n\nWhitepaper v 4.0\n\nPointPay \nMulti-currency Wallet\n\nPointPay \nCrypto Bank\n\nPointPay Crypto \nExchange platform\n\nPointPay \nOnline Payment System\n\nDiscovery.  Innovation.  Freedom\n\nhttps://pointpay.io/\nhttps://pointpay.io/\n\n\nTable of Contents \nExecutive Summary 3 \n\nIntroduction and Crypto-analytics from the PointPay platform 5 \n\nDisadvantages of Trading Crypto Ecosystem, or Money Must Work 6 \n\nThe Solution to the "Crypto Assets Depreciation" Problem 7 \n\nDisadvantages of Existing Crypto Banks. Why is the PointPay Crypto Bank the best? 8 \n\nBenefits of PointPay Crypto Bank over Conventional Banking 10 \n\nDescription of the PointPay Crypto Bank 12 \n\nPointPay Platform Investor Preferences 14 \n\nDisadvantages of Existing Crypto Exchanges. Why is the PointPay Exchange the Best? 15 \n\nCryptocurrency Exchange by PointPay. Centralized or Decentralized? 17 \n\nTokenized Assets Trading W

In [11]:
# Write to an excel file and save it
writer00 = pd.ExcelWriter(r'New_ICOdata150.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df1.to_excel(writer00)
writer00.close()

writer01 = pd.ExcelWriter(r'New_WPtext150.xlsx', engine='xlsxwriter')
wp_df1.to_excel(writer01)
writer01.close()

writer = pd.ExcelWriter(r'New_ICO_WPlinks150.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_wplinks1.to_excel(writer)
writer.close()

In [16]:
page_urls300 = get_page_links(150, 300)
data_ICO2, whtpapr_links2 = get_ico_info(page_urls300)

# Now for page 150 to 300


In [17]:
df_wplinks2 = pd.DataFrame(whtpapr_links2, columns = ['Coin name', "whitepaper link"]) 
df_wplinks2

,Coin name,whitepaper link
0,RankingBall,https://drive.google.com/file/d/1eFjLVu-pSY1-n...
1,Root Blockchain,https://rootblockchain.io/RootCoin.pdf
2,Safe Haven,https://safehaven.io/files/SafeHaven_WhitePape...
3,Stacktical,https://storage.googleapis.com/stacktical-publ...
4,TheFaustFlick,http://estudios-amazonia.com/TheFaustFlick_WP.pdf
5,Tokeneo,https://tokeneo.io/tokeneo_whitepaper.pdf
6,TV-TWO,https://tv-two.com/TV_Whitepaper.pdf
7,Viberate,https://www.viberate.io/Viberate_ICO_Whitepape...
8,VinChain,https://vinchain.io/files/VinChainWhitePaper.pdf
9,Waves,https://wavesplatform.com/files/images/whitepa...


In [19]:

df2 = pd.DataFrame(data_ICO2, columns=["CoinName","Category", "Rating","PreICO Price", "Price", "Softcap", "Hardcap", "Raised cap", "Whitepaper_link"] )
df2

,CoinName,Category,Rating,PreICO Price,Price,Softcap,Hardcap,Raised cap,Whitepaper_link
0,RankingBall,"Entertainment,Sports",4.1,"1 ETH = 50,000 RBG",0.0090 USD,"3,000,000 USD","30,000,000 USD",Unknown,https://drive.google.com/file/d/1eFjLVu-pSY1-n...
1,Root Blockchain,"Cryptocurrency,Investment,Media,Software",4.2,None,0.0300 USD,"1,000,000 USD","30,000,000 USD","$1,000,000",https://rootblockchain.io/RootCoin.pdf
2,Safe Haven,"Business services,Investment,Legal,Smart Contract",4.2,None,0.0030 USD,"3,000,000 USD","11,160,000 USD",Unknown,https://safehaven.io/files/SafeHaven_WhitePape...
3,Stacktical,"Internet,Infrastructure,Artificial Intelligenc...",4.3,1 DSLA = 0.000003 ETH,0.0004 USD,"1,000,000 EUR","3,000,000 EUR","$1,170,000",https://storage.googleapis.com/stacktical-publ...
4,TheFaustFlick,"Investment,Entertainment,Cryptocurrency,Media,...",3.9,1 TFF = 0.005 ETH,0.9571 USD,"11,000 ETH","20,000 ETH",Unknown,http://estudios-amazonia.com/TheFaustFlick_WP.pdf
5,Tokeneo,Investment,4.3,None,0.0275 USD,"3,000,000 USD","10,000,000 USD","$505,723",https://tokeneo.io/tokeneo_whitepaper.pdf
6,TV-TWO,"Entertainment,Platform,Media",4.1,None,0.0350 USD,"2,500 ETH","25,000 ETH","$8,700,000",https://tv-two.com/TV_Whitepaper.pdf
7,Viberate,Entertainment,4.1,None,0.1000 USD,"1,000,000 USD","12,000,000 USD","$10,714,28537,994.00000000 ETH",https://www.viberate.io/Viberate_ICO_Whitepape...
8,VinChain,"Business services,Big Data,Cryptocurrency,Infr...",4.0,None,0.0250 USD,"3,300 ETH","23,250 ETH","$6,197,044",https://vinchain.io/files/VinChainWhitePaper.pdf
9,Waves,"Business services,Platform",4.5,None,0.1880 USD,None,None,"$16,000,000",https://wavesplatform.com/files/images/whitepa...


In [20]:
writer10 = pd.ExcelWriter(r'New_ICOdata150_300.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df2.to_excel(writer10)
writer10.close()

writer = pd.ExcelWriter(r'New_ICO_WPlinks300.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_wplinks2.to_excel(writer)
writer.close()

In [22]:
whitepaper_list2 = get_wptext(whtpapr_links2)

2019-11-05 13:52:50,872 [MainThread  ] [INFO ]  Retrieving https://drive.google.com/file/d/1eFjLVu-pSY1-nDK7jXZ-phKzARWxLW0_/view to C:\Users\shiva\AppData\Local\Temp/https-drive-google-com-file-d-1efjlvu-psy1-ndk7jxz-phkzarwxlw0_-view.


GOT Whitepaper for: RankingBall 


2019-11-05 13:52:55,947 [MainThread  ] [INFO ]  Retrieving https://rootblockchain.io/RootCoin.pdf to C:\Users\shiva\AppData\Local\Temp/https-rootblockchain-io-rootcoin-pdf.


Cannot parse whitepaper of Root Blockchain at url:https://rootblockchain.io/RootCoin.pdf


2019-11-05 13:53:00,101 [MainThread  ] [INFO ]  Retrieving https://safehaven.io/files/SafeHaven_WhitePaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-safehaven-io-files-safehaven_whitepaper-pdf.


...

Cannot parse whitepaper of KredX at url:http://kredx.io/kredx.pdf


2019-11-05 18:36:53,734 [MainThread  ] [INFO ]  Retrieving https://kryptoro.com/images/whitepaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-kryptoro-com-images-whitepaper-pdf.
2019-11-05 18:36:56,952 [MainThread  ] [WARNI]  Tika server returned status: 503


Cannot parse whitepaper of Kryptoro at url:https://kryptoro.com/images/whitepaper.pdf


2019-11-05 18:37:00,954 [MainThread  ] [INFO ]  Retrieving https://kyber.network/assets/KyberNetworkWhitepaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-kyber-network-assets-kybernetworkwhitepaper-pdf.


Cannot parse whitepaper of KyberNetwork at url:https://kyber.network/assets/KyberNetworkWhitepaper.pdf


In [23]:
wp_df2 = pd.DataFrame(whitepaper_list2, columns=["CoinName", "Whitepaper Text"])
wp_df2

,CoinName,Whitepaper Text
0,RankingBall,Ranking-Ball-Gold-White-Paper.pdf - Google Dri...
1,Root Blockchain,No Access
2,Safe Haven,No Access
3,Stacktical,The future of downtime compensation \n\nAutoma...
4,TheFaustFlick,PRESENTS \n \n \n\n \n \n \n\n \n \n\nA WHITEP...
5,Tokeneo,No Access
6,TV-TWO,TV-TWO: Entertainment Ecosystem on the Blockch...
7,Viberate,A CROWDSOURCED \nLIVE MUSIC ECOSYSTEM \nAND A ...
8,VinChain,No Access
9,Waves,No Access


In [24]:
writer11 = pd.ExcelWriter(r'New_ICO_WPtext300.xlsx', engine='xlsxwriter')
wp_df2.to_excel(writer11)
writer11.close()

In [25]:
page_urls469 = get_page_links(301,469)
#Now for page 301 to 469

In [26]:
data_ICO3, whtpapr_links3 = get_ico_info(page_urls469)





In [28]:
df3 = pd.DataFrame(data_ICO3, columns=["CoinName","Category", "Rating","PreICO Price", "Price", "Softcap", "Hardcap", "Raised cap", "Whitepaper_link"] )
df3

,CoinName,Category,Rating,PreICO Price,Price,Softcap,Hardcap,Raised cap,Whitepaper_link
0,Kryptoro,"Cryptocurrency,Platform",2.9,None,1.0000 USD,"3,500,000 USD","35,000,000 USD",Unknown,https://kryptoro.com/images/whitepaper.pdf
1,KyberNetwork,Cryptocurrency,3.3,None,0.4100 USD,None,None,"$48,960,000200,000.00000000 ETH",https://kyber.network/assets/KyberNetworkWhite...
2,Lamoneda,Entertainment,2.9,1 LMDA = 0.00005 ETH,0.0195 USD,"2,000,000 USD","10,000,000 USD",Unknown,https://lamoneda.co/Whitepaper.pdf
3,Legolas Exchange,"Cryptocurrency,Investment,Platform",3.1,None,0.3325 USD,None,"1,000 BTC","$18,856,0003,504.00000000 BTC",https://legolas.exchange/static/public/legolas...
4,Lendoit,"Cryptocurrency,Internet,Banking,Platform",2.9,None,None,None,None,Unknown,https://lendoit.com/Lendoit-whitepaper.pdf
5,LevelApp,"Banking,Cryptocurrency",2.9,None,0.0079 USD,"700,000 USD","4,080,000 USD",Unknown,https://levelapp.cx/static/LevelApp_WhitePaper...
6,Lexzoom,Real estate,2.9,None,None,None,None,Unknown,https://www.lexzoom.tech/assets/lexzoom-whitep...
7,LIKE,Real estate,2.6,None,1.0000 USD,None,"45,000,000 USD",Unknown,http://likei.co/whitepaperen.pdf
8,LocalCoin.is DEX,"Banking,Cryptocurrency,Investment,Software",2.9,None,0.5000 USD,None,None,Unknown,https://localcoin.is/downloads/LocalCoinBlockc...
9,Luxreum,"Retail,Real estate,Platform,Other,Legal,Invest...",2.9,1 LXR = 0.1 USD,0.1000 USD,"2,000,000 USD","30,000,000 USD",Unknown,https://drive.google.com/file/d/1MqqNBCYwBb0v5...


In [30]:
df_wplinks3 = pd.DataFrame(whtpapr_links3, columns = ['Coin name', "whitepaper link"]) 
df_wplinks3

,Coin name,whitepaper link
0,Kryptoro,https://kryptoro.com/images/whitepaper.pdf
1,KyberNetwork,https://kyber.network/assets/KyberNetworkWhite...
2,Lamoneda,https://lamoneda.co/Whitepaper.pdf
3,Legolas Exchange,https://legolas.exchange/static/public/legolas...
4,Lendoit,https://lendoit.com/Lendoit-whitepaper.pdf
5,LevelApp,https://levelapp.cx/static/LevelApp_WhitePaper...
6,Lexzoom,https://www.lexzoom.tech/assets/lexzoom-whitep...
7,LIKE,http://likei.co/whitepaperen.pdf
8,LocalCoin.is DEX,https://localcoin.is/downloads/LocalCoinBlockc...
9,Luxreum,https://drive.google.com/file/d/1MqqNBCYwBb0v5...


In [31]:
writer20 = pd.ExcelWriter(r'New_ICOdata469.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df3.to_excel(writer20)
writer20.close()

writer = pd.ExcelWriter(r'New_ICO_WPlinks469.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_wplinks3.to_excel(writer)
writer.close()

In [33]:
whitepaper_list3 = get_wptext(whtpapr_links3)

2019-11-05 23:07:39,134 [MainThread  ] [INFO ]  Retrieving https://kryptoro.com/images/whitepaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-kryptoro-com-images-whitepaper-pdf.
2019-11-05 23:07:45,703 [MainThread  ] [WARNI]  Tika server returned status: 503


Cannot parse whitepaper of Kryptoro at url:https://kryptoro.com/images/whitepaper.pdf


2019-11-05 23:07:49,710 [MainThread  ] [INFO ]  Retrieving https://kyber.network/assets/KyberNetworkWhitepaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-kyber-network-assets-kybernetworkwhitepaper-pdf.


Cannot parse whitepaper of KyberNetwork at url:https://kyber.network/assets/KyberNetworkWhitepaper.pdf


2019-11-05 23:07:53,866 [MainThread  ] [INFO ]  Retrieving https://lamoneda.co/Whitepaper.pdf to C:\Users\shiva\AppData\Local\Temp/https-lamoneda-co-whitepaper-pdf.


...

Cannot parse whitepaper of Acme Network at url:http://www.acme.site/Acme_Whitepaper_V1.pdf


2019-11-06 05:15:29,894 [MainThread  ] [INFO ]  Retrieving https://docs.wixstatic.com/ugd/e5ff6c_3500b7cd9fce4d74aff59932d36727b9.pdf to C:\Users\shiva\AppData\Local\Temp/https-docs-wixstatic-com-ugd-e5ff6c_3500b7cd9fce4d74aff59932d36727b9-pdf.
2019-11-06 05:15:30,281 [MainThread  ] [WARNI]  Tika server returned status: 503


Cannot parse whitepaper of Credits at url:https://docs.wixstatic.com/ugd/e5ff6c_3500b7cd9fce4d74aff59932d36727b9.pdf


2019-11-06 05:15:34,283 [MainThread  ] [INFO ]  Retrieving https://docs.wixstatic.com/ugd/7c37b3_f2c27bc4258a48f4a0974a9a5418ba8b.docx?dn=%D0%B2%D0%B0%D0%B9%D1%82%20%D0%BF%D0%B5%D0%BF%D0%B5%D1%80.docx to C:\Users\shiva\AppData\Local\Temp/https-docs-wixstatic-com-ugd-7c37b3_f2c27bc4258a48f4a0974a9a5418ba8b-docx-dn-d0-b2-d0-b0-d0-b9-d1-82-20-d0-bf-d0-b5-d0-bf-d0-b5-d1-80-docx.
2019-11-06 05:15:34,435 [MainThread  ] [WARNI]  Tika server returned status: 503


Cannot parse whitepaper of Orada at url:https://docs.wixstatic.com/ugd/7c37b3_f2c27bc4258a48f4a0974a9a5418ba8b.docx?dn=%D0%B2%D0%B0%D0%B9%D1%82%20%D0%BF%D0%B5%D0%BF%D0%B5%D1%80.docx


In [34]:
wp_df3 = pd.DataFrame(whitepaper_list3, columns=["CoinName", "Whitepaper Text"])
wp_df3

,CoinName,Whitepaper Text
0,Kryptoro,No Access
1,KyberNetwork,No Access
2,Lamoneda,No Access
3,Legolas Exchange,No Access
4,Lendoit,No Access
5,LevelApp,No Access
6,Lexzoom,No Access
7,LIKE,No Access
8,LocalCoin.is DEX,No Access
9,Luxreum,No Access


In [35]:
writer21 = pd.ExcelWriter(r'New_ICO_WPtext469.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
wp_df3.to_excel(writer21)
writer21.close()

# created three excel files of the scraped data With coin details, whitepaper links and whitepaper text 
# In batches for Page 1 to 150, 151 to 300, 301 to 469.
#Total 5700 coins scraped.
